<span style="color:grey"> By Seongchan Kang </span>

<span style="color:grey"> Version : Python 3.10.1 in Window </span>

# Text Summarize 실습

아마존 리뷰 데이터를 통해 긴 텍스트를 문장에서 요약하는 딥러닝 모델을 만들기

트랜스포머(어택션)를 활용하여 적용

데이터는 캐글에서 가져옴 (URL : <span> https://www.kaggle.com/snap/amazon-fine-food-reviews </span>)

코드 관련 사이트 (URL : <span> https://wikidocs.net/72820 </span>)


# 대략적인 과정

1. 텍스트 데이터 가져오기

2. 텍스트 데이터 전처리  

3. 딥러닝 모델 설계

4. 어택션 메커니즘 적용

5. 평가 및 적용

# Seq2Seq

전체적인 과정을 들어가기 전에 Seq2Seq가 무엇인지 확인



## 데이터 가져오기

In [7]:
import numpy as np
import pandas as pd

In [8]:
review_df = pd.read_csv("Reviews.csv")
review_df

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...
...,...,...,...,...,...,...,...,...,...,...
568449,568450,B001EO7N10,A28KG5XORO54AY,Lettie D. Carter,0,0,5,1299628800,Will not do without,Great for sesame chicken..this is a good if no...
568450,568451,B003S1WTCU,A3I8AFVPEE8KI5,R. Sawyer,0,0,2,1331251200,disappointed,I'm disappointed with the flavor. The chocolat...
568451,568452,B004I613EE,A121AA1GQV751Z,"pksd ""pk_007""",2,2,5,1329782400,Perfect for our maltipoo,"These stars are small, so you can give 10-15 o..."
568452,568453,B004I613EE,A3IBEVCTXKNOH,"Kathy A. Welch ""katwel""",1,1,5,1331596800,Favorite Training and reward treat,These are the BEST treats for training and rew...


In [9]:
review_df = review_df[['Text','Summary']]
review_df

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy
...,...,...
568449,Great for sesame chicken..this is a good if no...,Will not do without
568450,I'm disappointed with the flavor. The chocolat...,disappointed
568451,"These stars are small, so you can give 10-15 o...",Perfect for our maltipoo
568452,These are the BEST treats for training and rew...,Favorite Training and reward treat


## 텍스트 데이터 전처리

### 중복된 데이터 처리

In [10]:
# 중복 확인
review_df.duplicated()

0         False
1         False
2         False
3         False
4         False
          ...  
568449    False
568450    False
568451    False
568452    False
568453    False
Length: 568454, dtype: bool

In [12]:
review_df = review_df.drop_duplicates(keep = 'first')
review_df

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy
...,...,...
568449,Great for sesame chicken..this is a good if no...,Will not do without
568450,I'm disappointed with the flavor. The chocolat...,disappointed
568451,"These stars are small, so you can give 10-15 o...",Perfect for our maltipoo
568452,These are the BEST treats for training and rew...,Favorite Training and reward treat


In [14]:
review_df = review_df.reset_index(drop = True)
review_df

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy
...,...,...
394965,Great for sesame chicken..this is a good if no...,Will not do without
394966,I'm disappointed with the flavor. The chocolat...,disappointed
394967,"These stars are small, so you can give 10-15 o...",Perfect for our maltipoo
394968,These are the BEST treats for training and rew...,Favorite Training and reward treat


### 결측치 데이터 처리

In [15]:
# 결측치 확인
review_df.isnull().sum()

Text       0
Summary    3
dtype: int64

In [16]:
# 결측치 제거
review_df = review_df.dropna()
review_df

,Text,Summary
0,I have bought several of the Vitality canned d...,Good Quality Dog Food
1,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised
2,This is a confection that has been around a fe...,"""Delight"" says it all"
3,If you are looking for the secret ingredient i...,Cough Medicine
4,Great taffy at a great price. There was a wid...,Great taffy
...,...,...
394965,Great for sesame chicken..this is a good if no...,Will not do without
394966,I'm disappointed with the flavor. The chocolat...,disappointed
394967,"These stars are small, so you can give 10-15 o...",Perfect for our maltipoo
394968,These are the BEST treats for training and rew...,Favorite Training and reward treat


In [17]:
# 결측치 재확인
review_df.isnull().sum()

Text       0
Summary    0
dtype: int64